# Google Drive

In [1]:
####################################
#
#  ADD THIS TO EVERY COLAB FILE!
#
####################################

from google.colab import drive
drive.mount('/content/drive')

import drive.Shareddrives.GPTJ.project.settings as settings

PATH_PROJECT = settings.PATH_PROJECT
PATH_DATA = settings.PATH_DATA

Mounted at /content/drive


In [2]:
! cd $PATH_PROJECT && pip install -q -r requirements.txt

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 143 kB 5.5 MB/s 
     |████████████████████████████████| 55.9 MB 1.2 MB/s 
     |████████████████████████████████| 85 kB 5.7 MB/s 
     |████████████████████████████████| 441 kB 63.5 MB/s 
     |████████████████████████████████| 163 kB 76.9 MB/s 
     |████████████████████████████████| 7.6 MB 50.8 MB/s 
     |████████████████████████████████| 1.3 MB 54.3 MB/s 
     |████████████████████████████████| 212 kB 64.7 MB/s 
     |████████████████████████████████| 115 kB 74.9 MB/s 
     |████████████████████████████████| 127 kB 69.6 MB/s 


In [3]:
!nvidia-smi

# Standard -> Tesla T4
# Premium -> Tesla P100-PCIE-16GB

Sun Oct 16 02:57:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import pandas as pd
import torch
torch.manual_seed(42)

from torch.utils.data import Dataset


# Dataset

In [5]:

class SimpleDataset(Dataset):

    def __init__(self, contents, labels, tokenizer, max_length=1024):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []

        for content, label in zip(contents, labels):

            encodings_dict = tokenizer(
                content,
                truncation=True,
                max_length=max_length, 
                padding='max_length'
            )

            input_ids = torch.tensor(encodings_dict['input_ids'])    
            self.input_ids.append(input_ids)

            mask = torch.tensor(encodings_dict['attention_mask'])
            self.attn_masks.append(mask)

            label = torch.tensor(label)
            self.labels.append(label)

            # print('item', len(input_ids), '-', input_ids, mask, label)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx], self.labels[idx]


# Treino

In [6]:
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel, GPT2ForSequenceClassification

model_name = 'gpt2'
labels = [5, 7]

tokenizer = GPT2Tokenizer.from_pretrained(
    model_name,
    num_labels = len(labels) # 0, 1
)
tokenizer.pad_token = tokenizer.eos_token

model = GPT2ForSequenceClassification.from_pretrained(model_name).cuda()
model.config.pad_token_id = model.config.eos_token_id

# model.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# cache
! mkdir -p /content/tmp/
! cp -r /content/drive/Shareddrives/GPTJ/data/kaggle/proc-1/5 /content/tmp/
! cp -r /content/drive/Shareddrives/GPTJ/data/kaggle/proc-1/7 /content/tmp/

In [8]:
torch.manual_seed(42)

from torch.utils.data import random_split

path_mask = '/content/tmp/{}/'
paths = [path_mask.format(i) for i in labels]

def load_dir(path):
    contents = []
    listdir = list(os.listdir(path))[:50]
    for filename in listdir:
        filepath = os.path.join(path, filename)
        with open(filepath, 'r') as f:
            contents.append(f.read())
    print(' ', len(contents), 'files')
    return contents

train_dataset = None
val_dataset = None
test_dataset = None

for i in range(len(labels)):
    print('load class', labels[i], 'as', i)
    contents = load_dir(paths[i])
    dataset_i = SimpleDataset(contents, [i]*len(contents), tokenizer)
    print('  len', len(contents))

    size_10 = int(0.1 * len(contents))
    size_80 = len(contents) - 2*size_10
    
    train_dataset_i, val_dataset_i, test_dataset_i = \
        random_split(dataset_i, [size_10, size_10, size_80])

    if not train_dataset:
        train_dataset = train_dataset_i
        val_dataset = val_dataset_i
        test_dataset = test_dataset_i
    else:
        train_dataset += train_dataset_i
        val_dataset += val_dataset_i
        test_dataset += test_dataset_i

print(train_dataset[-1])
print(val_dataset[-1])
print(test_dataset[-1])

load class 5 as 0
  42 files
  len 42
(tensor([  562,  2454, 50115,  ...,  1409,  1225,    87]), tensor([1, 1, 1,  ..., 1, 1, 1]), tensor(0))
(tensor([  562,  2454, 50115,  ...,  1225,    87,  3571]), tensor([1, 1, 1,  ..., 1, 1, 1]), tensor(0))
(tensor([  562,  2454, 50115,  ...,  4574,  3933,    71]), tensor([1, 1, 1,  ..., 1, 1, 1]), tensor(0))


In [ ]:
# (tensor([  562,  2454, 50115,  ...,  9940,    87,  3571]), tensor([1, 1, 1,  ..., 1, 1, 1]), tensor(1))
# (tensor([  562,  2454, 50115,  ...,  1409,  1225,    72]), tensor([1, 1, 1,  ..., 1, 1, 1]), tensor(1))
# (tensor([  562,  2454, 50115,  ...,    72,  3571, 21844]), tensor([1, 1, 1,  ..., 1, 1, 1]), tensor(1))

In [9]:
training_args = TrainingArguments(
    output_dir='/content/',
    num_train_epochs=2,
    # logging_steps=5000,
    # save_steps=5000,                                   
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=0,
    weight_decay=0.01,  
    # logging_dir=os.path.join(PATH_DATA, model_name, 'logs')
)

trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset, 
    # This custom collate function is necessary 
    # to built batches of data
    data_collator=lambda data: {
        'input_ids': torch.stack([f[0] for f in data]),       
        'attention_mask': torch.stack([f[1] for f in data]),
        'labels': torch.stack([f[2] for f in data])
    }
)
# Start training process!
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4
  Num Epochs = 2
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 4


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=4, training_loss=2.2335860194289126e-05, metrics={'train_runtime': 5.4255, 'train_samples_per_second': 1.475, 'train_steps_per_second': 0.737, 'total_flos': 4180748009472.0, 'train_loss': 2.2335860194289126e-05, 'epoch': 2.0})

# Classification

In [11]:
import numpy as np
from datasets import load_metric
metric = load_metric("accuracy")

test_predictions_weights = trainer.predict(test_dataset)
test_predictions = np.argmax(test_predictions_weights[0], axis=1)
print(test_predictions)

test_labels = [label for _, _, label in test_dataset]
print(test_labels)

test_references = np.array(test_labels)

metric.compute(predictions=test_predictions, references=test_references)
# {'accuracy': 0.91888}

***** Running Prediction *****
  Num examples = 34
  Batch size = 2


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0)]


{'accuracy': 1.0}